In [1]:
# open in Google Colab
# this is the BERT model we trained in class, now we just need the dataset

# the dataset is labeled, for us movie reviews that were nominated followed by 1
# then we need movie reviews that were not nominated, followed by 0
# probably want at least 6000 for our train, and 600 for our test, each split evenly with 1s and 0s


#Don't forget to create a folder in your Drive called allTwitterData for the Twitter dataset.

# mounting drive
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
# to install datasets and transformers
!pip install datasets
!pip install transformers

from datasets import load_dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.8 MB/s 
     |████████████████████████████████| 132 kB 84.7 MB/s 
     |████████████████████████████████| 182 kB 79.6 MB/s 
     |████████████████████████████████| 212 kB 70.4 MB/s 
     |████████████████████████████████| 127 kB 77.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.9 MB/s 
     |████████████████████████████████| 7.6 MB 50.6 MB/s 


In [3]:
# tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# preprocessing
def preprocess_function(examples):
  return tokenizer(examples["comments"], truncation=True)

# data collator with padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# training model
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

In [4]:
# function for computing accuracy
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
   load_precision = load_metric("precision")
   load_recall = load_metric("recall")
  
   logits, label = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=label)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=label)["f1"]
   precision = load_precision.compute(predictions=predictions, references=label)["precision"]
   recall = load_recall.compute(predictions=predictions, references=label)["recall"]
   return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [16]:
# loading in the files for train and test data
movieReviews = load_dataset('csv',data_files={'train': '/content/drive/MyDrive/Colab Notebooks/reddit_train.csv', 
                                           'test': '/content/drive/MyDrive/Colab Notebooks/reddit_test.csv'
                                           ,'predict': '/content/drive/MyDrive/Colab Notebooks/2022_reddit_no_titles.csv'
                                           })

# random samples from train and test data
small_train_dataset = movieReviews["train"].shuffle(seed=42).select([i for i in list(range(1000))])
small_test_dataset = movieReviews["test"].shuffle(seed=42).select([i for i in list(range(12))])
#small_predict_dataset = movieReviews["predict"].shuffle(seed=42).select([i for i in list(range(12))])

# tokenizing
tokenized_small_train = small_train_dataset.map(preprocess_function)
tokenized_small_test = small_test_dataset.map(preprocess_function)
tokenized_small_predict = movieReviews["predict"].map(preprocess_function)


Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating predict split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: ignored

In [14]:

# creaating output and hyperparameters
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/finalBERT/results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_small_train,
    eval_dataset=tokenized_small_test,
    #predict_dataset=tokenized_small_predict,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [17]:
# training
trainer.train()

# reporting metrics
trainer.evaluate()

predictions, labels, metrics = trainer.predict(tokenized_small_predict)

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: comments. If comments are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375
  Number of trainable parameters = 66955010


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: comments. If comments are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 12
  Batch size = 8


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: comments. If comments are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 19
  Batch size = 8
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
import pandas as pd

results = []

titles = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2022_reddit.csv")
for i in range(0, titles.shape[0]):
  if predictions[i][0] > predictions[i][1]:
    results.append([titles.loc[i][2], "Not Nominated"])
  else:
    results.append([titles.loc[i][2], "Nominated"])

for row in results:
  print(row)

#print(predictions)

['The Fabelmans', 'Nominated']
['Babylon', 'Not Nominated']
['Tar', 'Not Nominated']
['She Said', 'Not Nominated']
['The Woman King', 'Not Nominated']
['The Banshees of Inisherin', 'Nominated']
['Empire of Light', 'Nominated']
['Amsterdam', 'Not Nominated']
['Top Gun: Maverick', 'Nominated']
['Thirteen Lives', 'Not Nominated']
['Bullet Train', 'Not Nominated']
['Troll', 'Not Nominated']
['Violent Night', 'Not Nominated']
['The Menu', 'Not Nominated']
['Spirited', 'Not Nominated']
['X', 'Not Nominated']
['Black Panther: Wakanda Forever', 'Not Nominated']
['Pinnochio', 'Not Nominated']
['Emancipation', 'Not Nominated']
